In [1]:
#23/8/24 Creates a K fold KNN model for each file in a folder provided they are chemopy

import os
import pandas as pd
import numpy as np
import math
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Targets/Drugbank/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = df.drop(['Unnamed: 0', 'inchi'], axis = 1)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
splitcount = 3
resultslist = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / splitcount
    splitsize = math.ceil(size)
    empty = [[] for _ in range(splitcount)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,splitcount):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 4):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = mtry,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)


            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = bestmetrics,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)

        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 2 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    resultslist.append(results[12])

For dataset Cardiotoxicity_Cardiotoxicity-1_DBtargets.csv
for fold 1 test set mcc of 0 valid set mcc of 0.4403855060505443
for fold 2 test set mcc of 0 valid set mcc of 0.17766726362967536
for fold 3 test set mcc of 0 valid set mcc of 0

validation metrics of:
positives in data 0
negatives in data 0
net accuracy = 0
mcc = 0
For dataset Endocrine Disruption_SR-ARE_DBtargets.csv
for fold 1 test set mcc of 0.4062025678767146 valid set mcc of 0.21604339257368527
for fold 2 test set mcc of 0 valid set mcc of 0.0890870806374748
for fold 3 test set mcc of 0.13137909646792317 valid set mcc of 0.1829534767323246

validation metrics of:
positives in data 37
negatives in data 53
net accuracy = 0.6222222222222222
mcc = 0.19169235233734833
For dataset Cardiotoxicity_Cardiotoxicity-5_DBtargets.csv
for fold 1 test set mcc of 0.19672236884115843 valid set mcc of 0.17541160386140586
for fold 2 test set mcc of 0.3931785497463923 valid set mcc of 0.23904572186687875
for fold 3 test set mcc of 0.388443293

In [7]:
namelist

['NR-ER_Chemopy.csv',
 'Developmental_Toxicity_Chemopy.csv',
 'NR-aromatase_Chemopy.csv',
 'SR-ATAD5_Chemopy.csv',
 'SR-ARE_Chemopy.csv',
 'SR-p53_Chemopy.csv',
 'Cardiotoxicity-1_Chemopy.csv',
 'SR-HSE_Chemopy.csv',
 'Respiratory_Toxicity_Chemopy.csv',
 'Carcinogenicity_Chemopy.csv',
 'Cardiotoxicity-30_Chemopy.csv',
 'Cardiotoxicity-10_Chemopy.csv',
 'Cardiotoxicity-5_Chemopy.csv',
 'NR-AR-LBD_Chemopy.csv',
 'Ames_Mutagenicity_Chemopy.csv',
 'Hepatotoxicity_Chemopy.csv',
 'NR-AR_Chemopy.csv',
 'Reproductive_Toxicity_Chemopy.csv',
 'NR-ER-LBD_Chemopy.csv',
 'NR-PPAR-gamma_Chemopy.csv',
 'NR-AhR_Chemopy.csv',
 'SR-MMP_Chemopy.csv']

In [3]:
len(resultslist)

19

In [2]:
temp = pd.DataFrame(resultslist, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/DrugBank Targets/modelout/drugbank_svm')